# I. Introduction

This is ou first on a competition as Team. The goal of this notebook is to find a correlation between particular attributes of products and the number of sales.

## 1.1. Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix 

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from itertools import product
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Looking quickly through the csv, we see that only 3 will be usefull.

In [ ]:
ds_items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
ds_sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
ds_submit = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
ds_items.columns, ds_sales_train.columns, ds_submit.columns

## 1.2. Creating the datasets

### 1.2.1. Handling outliers

In [ ]:
ds_train = ds_sales_train[ds_sales_train["item_cnt_day"] < 1500]
ds_train = ds_train[ds_train["item_price"] < 75000]

In [ ]:
ds_train = ds_train[['date_block_num', 'shop_id', 'item_id','item_cnt_day']]
ds_train = ds_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index = False).sum()
ds_train.rename(columns={"item_cnt_day": 'item_cnt_month'}, inplace = True)

### 1.2.2. Data Augmentation

For each **id** and **shop_id** and **date_block_num** combinations we complete the dataset by putting it to 0 if it isn't in the dataset.

In [ ]:
ds_train

In [ ]:
ds_train = ds_train[ds_train['shop_id'].isin(ds_submit['shop_id'].unique())]
ds_train = ds_train[ds_train['item_id'].isin(ds_submit['item_id'].unique())]

In [ ]:
%%time
ds_combinations = []
cols  = ["date_block_num", "shop_id", "item_id"]
ds_combinations.append(np.array(list(product(ds_train["date_block_num"].unique(),ds_train["shop_id"].unique(),ds_train["item_id"].unique()))))
ds_combinations = pd.DataFrame(np.vstack(ds_combinations), columns = cols)

In [ ]:
ds_train = pd.merge(ds_combinations,ds_train, how='left', on=cols)

In [ ]:
ds_train.fillna(0, inplace=True)
ds_train

### 1.2.3. Handling columns (join)

In [ ]:
ds_train = ds_train.join(ds_items.set_index('item_id'), on="item_id").drop(["item_name"], axis=1)
ds_submit = ds_submit.join(ds_items.set_index('item_id'), on="item_id").drop(["item_name"], axis=1)

In [ ]:
ds_submit.set_index("ID", inplace= True)
ds_submit['date_block_num'] = 34

In [ ]:
ds_train

In [ ]:
ds_submit

# 2. Preprocessing

## 2.1. One hot encoding

We first decided to use one hot encoding on our categorical data but the high number of distinct categories creates a high dimensional matrix which creates a way too difficult problem to solve for models.
We will keep our former work and make it as comments.

### 2.2.1. Months

In [ ]:
#Months = {0:"January", 1:"February", 2:"March", 3:"April", 4:"May", 5:"June", 6:"July", 7:"August", 8:"September", 9:"October", 10:"November", 11:"December"}
#one_hot = pd.get_dummies(ds_train['month'], sparse=True)
#ds_train = ds_train.drop('month',axis = 1)
#ds_train = ds_train.join(one_hot)
#ds_train.rename(columns=Months, inplace=True)

In [ ]:
#one_hot = pd.get_dummies(ds_submit['month'], sparse=True)
#ds_submit = ds_submit.drop('month',axis = 1)
#ds_submit = ds_submit.join(one_hot)
#ds_submit.rename(columns=Months, inplace=True)

### 2.2.2. Shop_id

In [ ]:
#one_hot = pd.get_dummies(ds_train['shop_id'], prefix='shop_', sparse=True)#
#ds_train = ds_train.drop('shop_id',axis = 1)
#ds_train = ds_train.join(one_hot)

In [ ]:
#one_hot = pd.get_dummies(ds_submit['shop_id'], prefix='shop_', sparse=True)
#ds_submit = ds_submit.drop('shop_id',axis = 1)
#ds_submit = ds_submit.join(one_hot)

### 2.2.3. Item categories

In [ ]:
#one_hot = pd.get_dummies(ds_train['item_category_id'], prefix='item_category_', sparse=True)
#ds_train = ds_train.drop('item_category_id',axis = 1)
#ds_train = ds_train.join(one_hot)

In [ ]:
#one_hot = pd.get_dummies(ds_submit['item_category_id'], prefix='item_category_', sparse=True)
#ds_submit = ds_submit.drop('item_category_id',axis = 1)
#ds_submit = ds_submit.join(one_hot)

### 2.2.3. Item IDs

In [ ]:
#one_hot = pd.get_dummies(ds_train['item_id'], prefix='item_id_', sparse=True)#
#ds_train = ds_train.drop('item_id',axis = 1)
#ds_train = ds_train.join(one_hot)

In [ ]:
#one_hot = pd.get_dummies(ds_submit['item_id'], prefix='item_id_', sparse=True)##
#ds_submit = ds_submit.drop('item_id',axis=1)
#ds_submit = ds_submit.join(one_hot)

## 2.4. Feature Engineering

### 2.4.1. Making sense of months

In [ ]:
ds_train["month"] = (ds_train["date_block_num"] % 12).astype('category')
ds_submit["month"] = (ds_submit["date_block_num"] % 12).astype('category')

In [ ]:
ds_train["year"] = (ds_train["date_block_num"] // 12).astype('category')
ds_submit["year"] = (ds_submit["date_block_num"] // 12).astype('category')

### 2.4.2. Adding a column lag on months

In [ ]:
tmp = ds_train[['date_block_num','shop_id','item_id','item_cnt_month']]
shifted = tmp.copy()
tmp.columns = ['date_block_num','shop_id','item_id', 'item_cnt_month'+'_lag_'+str(1)]
tmp['date_block_num'] += 1
ds_train = pd.merge(ds_train, tmp, on=['date_block_num','shop_id','item_id'], how='left')
ds_train['item_cnt_month'+'_lag_'+str(1)] = ds_train['item_cnt_month'+'_lag_'+str(1)].astype('float16')
ds_train.fillna(0, inplace=True)

In [ ]:
tmp = ds_train[['date_block_num','shop_id','item_id','item_cnt_month']]
tmp.columns = ['date_block_num','shop_id','item_id', 'item_cnt_month'+'_lag_'+str(1)]
tmp['date_block_num'] += 1
ds_submit = pd.merge(ds_submit, tmp, on=['date_block_num','shop_id','item_id'], how='left')
ds_submit['item_cnt_month'+'_lag_'+str(1)] = ds_submit['item_cnt_month'+'_lag_'+str(1)].astype('float16')
ds_submit = ds_submit.replace(np.nan, 0)

### 2.4.3. Adding holiday/week days counts and the first interaction of products

In [ ]:
def count_holidays(date_block_num):
    m = 1 + date_block_num % 12
    if m == 1:
        return 1
    elif m == 2:
        return 1
    elif m == 3:
        return 1
    elif m == 5:
        return 2
    elif m == 6:
        return 1
    elif m == 11:
        return 1
    elif m == 12:
        return 2
    else:
        return 0
    
def count_weekdays(date_block_num):
    try:
        y = 2013 + date_block_num // 12
        m = 1 + date_block_num % 12
        if m < 9:
            return np.busday_count(f'{y}-0{m}', f'{y}-0{m+1}')
        elif m ==9:
            return np.busday_count(f'{y}-0{m}', f'{y}-10')
        elif m != 12:
             return np.busday_count(f'{y}-{m}', f'{y}-{m+1}')
        else:
            return np.busday_count(f'{y}-{m}', f'{y+1}-01')
    except ValueError:
        print(m,y)

In [ ]:
%%time
ds_train["holidays_in_month"] = ds_train["date_block_num"].apply(lambda x: count_holidays(x))
ds_train["weekdays_in_month"] = ds_train["date_block_num"].apply(lambda x: count_weekdays(x))

ds_submit["holidays_in_month"] = ds_submit["date_block_num"].apply(lambda x: count_holidays(x))
ds_submit["weekdays_in_month"] = ds_submit["date_block_num"].apply(lambda x: count_weekdays(x))

In [ ]:
group = ds_train.groupby(['item_id'])['date_block_num'].min().reset_index()
group.rename(columns={"date_block_num": "item_first_interaction"}, inplace=True)
ds_train = pd.merge(ds_train, group, on=['item_id'], how='left')
ds_train["item_first_interaction"] = ds_train["date_block_num"] - ds_train["item_first_interaction"]

group = ds_submit.groupby(['item_id'])['date_block_num'].min().reset_index()
group.rename(columns={"date_block_num": "item_first_interaction"}, inplace=True)
ds_submit = pd.merge(ds_submit, group, on=['item_id'], how='left')
ds_submit["item_first_interaction"] = ds_submit["date_block_num"] - ds_submit["item_first_interaction"]

### 2.4.4. Handle duplicates

In [ ]:
# Якутск Орджоникидзе, 56

ds_train[ds_train['shop_id'] == 0]['shop_id'] = 57
ds_submit[ds_submit['shop_id'] == 0]['shop_id'] = 57

# Якутск ТЦ "Центральный"

ds_train[ds_train['shop_id'] == 1]['shop_id'] = 58
ds_submit[ds_submit['shop_id'] == 1]['shop_id'] = 58

# Жуковский ул. Чкалова 39м²

ds_train[ds_train['shop_id'] == 10]['shop_id'] = 11
ds_submit[ds_submit['shop_id'] == 10]['shop_id'] = 11

## 2.5. Making the test set and data set coherent

Now we must complete the test set with the columns from the train set.

In [ ]:
for col in ds_train.columns:
    if col not in ds_submit.columns:
        if col != "item_cnt_month":
            null_column = np.zeros(ds_submit.shape[0], dtype=int)
            ds_submit[col] = pd.arrays.SparseArray(null_column)

In [ ]:
for col in ds_submit.columns:
    if col not in ds_train.columns:
        null_column = np.zeros(ds_train.shape[0], dtype=int)
        ds_train[col] = pd.arrays.SparseArray(null_column) 

In [ ]:
ds_train.shape[1]==ds_submit.shape[1]
ds_train

In [ ]:
ds_train['date_block_num'] = ds_train['date_block_num']
ds_train['year'] = ds_train['year']
ds_train['item_cnt_month_lag_1'] = ds_train['item_cnt_month_lag_1']

# 3. Modeling

## 3.1. Creating sets

For our test set, it makes sense to guess the 33rd month using the previous ones as we want to predict the 34th one for the submission.

In [ ]:
for col in ds_submit.columns:
    ds_submit[col] = ds_submit[col].astype("Sparse[int]")

for col in ds_train.columns:
    ds_train[col] = ds_train[col].astype("Sparse[int]")

In [ ]:
ds_test = ds_train[ds_train['date_block_num'] == 33]
y_test = ds_test.pop('item_cnt_month').to_numpy()
X_test  = ds_test.sparse.to_coo().tocsr()
X_test

In [ ]:
ds_trn = ds_train[ds_train['date_block_num'] != 33].copy()
y_train = ds_trn.pop('item_cnt_month').to_numpy()
X_train = ds_trn.sparse.to_coo().tocsr()
X_train

In [ ]:
ds_submit['date_block_num'] = ds_submit['date_block_num'].astype('Sparse[int]')
ds_submit['year'] = ds_test['year'].astype('Sparse[int]')
ds_submit['item_cnt_month_lag_1'] = ds_submit['item_cnt_month_lag_1'].astype('Sparse[int]')

X_submit = ds_submit.sparse.to_coo().tocsr()

## 3.2. Models

In [ ]:
def evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(y_pred)
    print(mean_squared_error(y_test, y_pred))

### 3.2.1. XGBRegressor

In [ ]:
X_train

In [ ]:
%%time 
mse = 100
for learning_rate in [0.05,0.1, 0.5,1]:
    for max_depth in [5, 10, 25,50]:
            print(learning_rate,max_depth)
            model = XGBRegressor(
                learning_rate = learning_rate,
                max_depth=max_depth,
                booster='gbtree',
                n_estimators=1000,
                min_child_weight=0.5, 
                subsample=0.8,
                sampling_method="uniform",
                colsample_bynode=1,
                colsample_bytree=0.8, 
                eta=0.1)

            model.fit(
                X_train, 
                y_train, 
                eval_metric="rmse", 
                eval_set=[(X_train, y_train), (X_test, y_test)], 
                verbose=True, 
                early_stopping_rounds = 20)

            y_pred = model.predict(X_test)
            if mse>mean_squared_error(y_test, y_pred):
                lr = learning_rate
                mse = mean_squared_error(y_test, y_pred)
                mx_d = max_depth
                best_iter = model.get_booster().best_iteration
            print('XGBoost MSE =', mean_squared_error(y_test, y_pred), "\n")

In [ ]:
print(f"Best model had parameters : learning_rate = {lr} and max_depth={mx_d} | MSE = {mse}")

## 3.3. Recreate the full dataset and get prediction

In [ ]:
ds_train

In [ ]:
y_train = ds_train.pop('item_cnt_month').to_numpy()
X_train = ds_train.sparse.to_coo().tocsr()
X_train

In [ ]:
%%time
model = XGBRegressor(
    learning_rate=lr,
    max_depth=mx_d,
    booster='gbtree',
    n_estimators=best_iter,
    min_child_weight=0.5, 
    subsample=0.8,
    sampling_method="uniform",
    colsample_bynode=1,
    colsample_bytree=0.8, 
    eta=0.1)

model.fit(X_train,y_train,
    verbose=True)

In [ ]:
feature_importance = pd.DataFrame(model.feature_importances_, index=ds_train.columns)
feature_importance[feature_importance[0] > 0.005].plot.bar(figsize=(15,6))
plt.show()

In [ ]:
ds_train.columns

In [ ]:
ds_submit.columns

In [ ]:
y_submit = model.predict(X_submit)
ds_submit = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
ds_submit['item_cnt_month'] = np.clip(y_submit,0,20) 
ds_submit.drop(["shop_id","item_id"], inplace=True, axis=1)

In [ ]:
ds_submit.to_csv('/kaggle/working/submission.csv', index=False)